In [10]:

import numpy as np

# Orbital parameters (from literature for Alpha Centauri AB)
semi_major_axis_arcsec = 17.493  # Semi-major axis in arcseconds
eccentricity = 0.52  # Orbital eccentricity
inclination_deg = 79.2  # Inclination in degrees
omega_deg = 231.65  # Longitude of periastron in degrees
epoch_periastron = 1875.66  # Epoch of periastron in Julian years
orbital_period_years = 79 # Orbital period in years


target_year = 2026

# Convert degrees to radians for inclination and omega
inclination_rad = np.deg2rad(inclination_deg)
omega_rad = np.deg2rad(omega_deg)

# Time difference from periastron to target year
time_since_periastron_years = target_year - epoch_periastron

# Mean anomaly (M) in radians
mean_anomaly_rad = 2 * np.pi * (time_since_periastron_years % orbital_period_years) / orbital_period_years

# Solve Kepler's equation: M = E - e * sin(E) for E (Eccentric Anomaly)
def solve_keplers_equation(M, e, tol=1e-6):
    E = M  # Initial guess
    while True:
        delta_E = (M - (E - e * np.sin(E))) / (1 - e * np.cos(E))
        E += delta_E
        if abs(delta_E) < tol:
            break
    return E

eccentric_anomaly_rad = solve_keplers_equation(mean_anomaly_rad, eccentricity)

# True anomaly (ν) in radians
true_anomaly_rad = 2 * np.arctan2(
    np.sqrt(1 + eccentricity) * np.sin(eccentric_anomaly_rad / 2),
    np.sqrt(1 - eccentricity) * np.cos(eccentric_anomaly_rad / 2)
)

# Distance from the center of mass (r) in arcseconds
r_arcsec = semi_major_axis_arcsec * (1 - eccentricity**2) / (1 + eccentricity * np.cos(true_anomaly_rad))

# Projected separation in arcseconds (accounting for inclination and ω)
projected_separation_arcsec = r_arcsec * np.sqrt(
    (np.sin(true_anomaly_rad + omega_rad))**2 +
    (np.cos(true_anomaly_rad + omega_rad) * np.cos(inclination_rad))**2
)

print("Binary separation between Alpha Cen in 2026 (in arcseconds):",projected_separation_arcsec)


Binary separation between Alpha Cen in 2026 (in arcseconds): 8.77477687328834


Explanation:
This code calculates the projected separation in arcseconds of Alpha Centauri AB for a given target year (2026) based on its orbital parameters. It first computes the mean anomaly, which represents the fraction of the orbital period elapsed since the last periastron, assuming uniform circular motion. Using Kepler's equation, it then solves for the eccentric anomaly, which accounts for the elliptical nature of the orbit and maps the body's position onto a reference circle. From this, the true anomaly is determined, representing the actual angular position of the body along its elliptical orbit. These anomalies describe the body's orbital dynamics at different levels of precision. The orbital radius at this point is calculated, and the projected separation is derived by accounting for the inclination and orientation of the orbit. Key calculations involve trigonometric transformations and Keplerian orbital mechanics. The final output is the apparent distance between the stars as seen from Earth.

References:
Eccentricity (e = 0.52):https://www.aanda.org/articles/aa/full_html/2016/10/aa29201-16/aa29201-16.html
inclination 79.2: https://www.aanda.org/articles/aa/pdf/2016/10/aa29201-16.pdf
omega_deg = 231.65 :https://www.mdpi.com/2218-1997/10/2/64
epoch_periastron = 1875.66:https://en.wikipedia.org/wiki/Alpha_Centauri
semi major axis=17.393arcseconds and orbital period=79 years: https://en.wikipedia.org/wiki/Alpha_Centauri

In [13]:
from astroquery.simbad import Simbad


Simbad.add_votable_fields('flux(V)', 'fe_h', 'sp', 'rv_value')

# Query for Alpha Centauri A
result_A = Simbad.query_object("Alpha Cen A")
print(result_A)

# Extracting values if needed
V_mag = result_A['FLUX_V'][0]
print(f"Alpha Cen A: V Magnitude: {V_mag}")

# Spectral Type
spectral_type = result_A['SP_TYPE'][0]
print(f"Alpha Cen A: Spectral Type: {spectral_type}")

# Metallicity
metallicity = result_A['FE_H'][0] if 'FE_H' in result_A.colnames else "Not Available"
print(f"Alpha Cen A: Metallicity [Fe/H]: {metallicity}")

# Radial Velocity
radial_velocity = result_A['RV_VALUE'][0]
print(f"Alpha Cen A Radial Velocity: {radial_velocity} km/s")





  MAIN_ID         RA           DEC      ... RV_VALUE_4 SCRIPT_NUMBER_ID
               "h:m:s"       "d:m:s"    ...   km / s                   
----------- ------------- ------------- ... ---------- ----------------
* alf Cen A 14 39 36.4940 -60 50 02.373 ...    -15.252                1
Alpha Cen A: V Magnitude: 0.009999999776482582
Alpha Cen A: Spectral Type: G2V
Alpha Cen A: Metallicity [Fe/H]: Not Available
Alpha Cen A Radial Velocity: -15.252 km/s


In above code, V magnitude and Effective temperature of Alpha Centauri A and B has been taken from https://adsabs.harvard.edu/full/1986A%26A...158..273S .TOLIMAN bandpass 530-640 nm has been taken from https://doi.org/10.1117/12.3019202

Here, I take Primary mirror diameter (13 cm) and secondary mirror diamater (1.90 cm) and spider_fraction = 0.0214 

Here, I start by modeling Alpha Centauri A's spectral flux density as a blackbody with a temperature of 5781 K and integrating it over the TOLIMAN bandpass (530-640 nm) to determine the total emitted flux. This flux is then normalized using the V-band magnitude (-0.1) and the Vega system zero-point flux to match observed brightness. Next, the effective collecting area of a TOLIMAN telescope is calculated, accounting for losses due to the secondary mirror and spider arm obstructions. The integrated flux is scaled by this effective area, providing an estimate of the light actually collected by the telescope. I also divide the collected flux (in erg/s) by the energy of a single photon (in erg) is the correct approach to convert it into photons per second. 

In [ ]:
from astroquery.simbad import Simbad


Simbad.add_votable_fields('flux(V)', 'fe_h', 'sp', 'rv_value')

# Query for Alpha Centauri B
result_A = Simbad.query_object("Alpha Cen B")
print(result_A)


V_mag = result_A['FLUX_V'][0]
print(f"Alpha Cen B: V Magnitude: {V_mag}")

# Spectral Type
spectral_type = result_A['SP_TYPE'][0]
print(f"Alpha Cen B: Spectral Type: {spectral_type}")

# Metallicity
metallicity = result_A['FE_H'][0] if 'FE_H' in result_A.colnames else "Not Available"
print(f"Metallicity [Fe/H]: {metallicity}")

# Radial Velocity
radial_velocity = result_A['RV_VALUE'][0]
print(f"Alpha Cen B:Radial Velocity: {radial_velocity} km/s")


  MAIN_ID         RA           DEC      ... RV_VALUE_2 SCRIPT_NUMBER_ID
               "h:m:s"       "d:m:s"    ...   km / s                   
----------- ------------- ------------- ... ---------- ----------------
* alf Cen B 14 39 35.0631 -60 50 15.099 ...    -22.586                1
Alpha Cen B: V Magnitude: 1.3300000429153442
Alpha Cen B: Spectral Type: K1V
Metallicity [Fe/H]: Not Available
Alpha Cen B:Radial Velocity: -22.586 km/s


Metallicity   [Fe/H] of alpha centurai  ~ +0.23 dex 
Ref: https://www.aanda.org/articles/aa/full_html/2018/07/aa33125-18/aa33125-18.html#F2

Key points:
*** The change in position of Alpha Centauri AB system in 2026 is approximately 7.400000 arcseconds.
Converted to light-years, this change in separation is approximately 0.000157 light-years.






***alf Cen A sky position: 14 39 36.4940 -60 50 02.373             1
Alpha Cen A: V Magnitude: 0.009999999776482582
Alpha Cen A: Spectral Type: G2V
Alpha Cen A Radial Velocity: -15.252 km/s

***alf Cen B sky position:14 39 35.0631 -60 50 15.099       1
Alpha Cen B: V Magnitude: 1.3300000429153442
Alpha Cen B: Spectral Type: K1V
Alpha Cen B:Radial Velocity: -22.586 km/s

